# Automated ML

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core.dataset import Dataset
from azureml.train.automl.utilities import get_primary_metrics
from azureml.train.automl import AutoMLConfig
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.automl.core.shared import constants
import joblib

## Dataset

### Overview

The dataset used for this project is the [Heart Failure Prediction](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) dataset taken from Kaggle.

This dataset contains 12 features that can be used to predict mortality by heart failure:

- age: Age of the patient
- amaemia: Decrease of red blood cells or hemoglobin
- creatinine_phosphokinase: Level of the CPK enzyme in the blood (mcg/L)
- diabetes: If the patient has diabetes
- ejection_fraction: Percentage of blood leaving the heart at each contraction
- high_blood_pressure: If the patient has hypertension
- platelets: Platelets in the blood (kiloplatelets/mL)
- serum_creatinine: Level of serum creatinine in the blood (mg/dL)
- serum_sodium: Level of serum sodium in the blood (mEq/L)
- sex: Woman or man
- smoking: If the patient smokes or not
- time: Follow-up period (days)

The target column is DEATH_EVENT which tells if the patient deceased during the follow-up period. The task performed in this project is to predict whether or not a death event occurs.

In [2]:
## Creating a new Experiment
ws = Workspace.from_config()
experiment_name = 'heart-failure-automl'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-138577
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-138577


In [3]:
#Checking and printing existing compute targets
compute_targets= ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

notebook138577 ComputeInstance Succeeded


In [4]:
#Create compute cluster
compute_cluster_name= "cpu-compute"

#Check if compute cluster already exists
try:
    compute_cluster=ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, use it...")
except ComputeTargetException:
    print("Creating new cluster...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=5)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    
compute_cluster.wait_for_completion(show_output=True)

Creating new cluster...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
#Get data
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://raw.githubusercontent.com/neha7598/azure-ml-capstone/main/data/heart_failure_clinical_records_dataset.csv"

path_to_data= "https://raw.githubusercontent.com/neha7598/azure-ml-capstone/main/data/heart_failure_clinical_records_dataset.csv"
data=TabularDatasetFactory.from_delimited_files(path=path_to_data)

In [6]:
data = data.to_pandas_dataframe()
x=data.drop('DEATH_EVENT',axis=1)
y=data['DEATH_EVENT']

##split into train and test datasets
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.20)

#concatenate to form train and test datasets 
train_df=pd.concat([x_train, y_train], axis=1)
test_df=pd.concat([x_test, y_test], axis=1)

train_df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
285,55.0,1,170,1,40,0,336000.00,1.20,135,1,0,250,0
166,53.0,0,196,0,60,0,220000.00,0.70,133,1,1,134,0
174,65.0,0,198,1,35,1,281000.00,0.90,137,1,1,146,0
102,80.0,0,898,0,25,0,149000.00,1.10,144,1,1,87,0
220,73.0,0,582,0,20,0,263358.03,1.83,134,1,0,198,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,55.0,0,84,1,38,0,451000.00,1.30,136,0,0,246,0
115,58.0,1,400,0,40,0,164000.00,1.00,139,0,0,91,0
158,85.0,1,910,0,50,0,235000.00,1.30,134,1,0,121,0
24,75.0,0,582,1,30,1,263358.03,1.83,134,0,0,23,1


In [7]:
#save pandas dataframe as .csv and upload to datastore
if not os.path.isdir('data'):
    os.mkdir('data')
pd.DataFrame(train_df).to_csv("data/train_data.csv", index=False)
pd.DataFrame(test_df).to_csv("data/test_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='heart-failure', overwrite=True, show_progress=True)

Uploading an estimated of 2 files
Uploading ./data/test_data.csv
Uploaded ./data/test_data.csv, 1 files out of an estimated total of 2
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_3e0ea4b9875a4793b3a948812510aa6c

In [8]:
#Load dataset as TabularDataset
train_data = Dataset.Tabular.from_delimited_files(path=ds.path('heart-failure/train_data.csv'))

In [9]:
get_primary_metrics("classification")

['norm_macro_recall',
 'AUC_weighted',
 'precision_score_weighted',
 'accuracy',
 'average_precision_score_weighted']

## AutoML Configuration
The AutoML Config class is a way of leveraging the AutoML SDK to automate machine learning. 

The AutoML settings file is a dictionary that specifies all the parameters controlling the experiment like experiment_timeout_minutes, whether or not to enable early stoppong, number of cross validations, the primary metric, etc.

In the AutoML Config, we specify the task, the training data to be used, the target column name, the compute target and we pass the automl settings dictionary.

In [11]:
# automl settings 
automl_settings = {
    "enable_early_stopping" : True,
    "experiment_timeout_minutes": 30,
    "n_cross_validations": 4,
    "featurization": 'auto',
    "primary_metric": 'accuracy',
    "verbosity": logging.INFO
}

# automl config 
automl_config = AutoMLConfig(
    task='classification',
    debug_log = 'automl_errors.log',
    training_data=train_data,
    label_column_name='DEATH_EVENT',
    compute_target=compute_cluster,
    **automl_settings
)

In [12]:
# Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

In [13]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [14]:
remote_run.wait_for_completion()

{'runId': 'AutoML_3e325779-ff8d-4e8c-8f0b-9a482989a051',
 'target': 'cpu-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-02-11T14:14:54.245623Z',
 'endTimeUtc': '2021-02-11T14:56:21.589808Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'cpu-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-failure-automl","subscription_id":"510b94ba-e453-4417-988b-fbdc37b55ca7","resource_group":"aml-quickstarts-138577","workspace_name":"quick-starts-ws-138577","region":"southcentralus","compute_target":"cpu-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":4,"y_min":null,"y_max":null,"num_classes":

## Best Model

In [15]:
#get best model and print all the metrics
best_run, fitted_model = remote_run.get_output()

best_run_metrics = best_run.get_metrics() 
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


recall_score_weighted 0.887005649717514
f1_score_macro 0.8740990734327544
average_precision_score_weighted 0.9316850633756819
norm_macro_recall 0.7508942927347934
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_3e325779-ff8d-4e8c-8f0b-9a482989a051_26/confusion_matrix
average_precision_score_micro 0.9387781197144919
f1_score_micro 0.887005649717514
precision_score_weighted 0.8895307169599654
accuracy 0.887005649717514
AUC_weighted 0.9298096362642255
precision_score_micro 0.887005649717514
f1_score_weighted 0.8869402000645283
balanced_accuracy 0.8754471463673967
average_precision_score_macro 0.9153582497473376
AUC_macro 0.9298096362642255
recall_score_macro 0.8754471463673967
matthews_correlation 0.7512721116896034
AUC_micro 0.9362886223626671
recall_score_micro 0.887005649717514
weighted_accuracy 0.8960937904397048
precision_score_macro 0.8762014411027568
log_loss 0.34071214863878496
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_3e325779-ff8d-4e8c-8f0b-9a482989a0

In [20]:
# Details of best model as well as the parameters of the best run
fitted_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               objective='reg:logistic',
                                                                                               random_state=0,
                                                                                               reg_alpha=2.0833333333333335,
                                   

In [19]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-automl,AutoML_3e325779-ff8d-4e8c-8f0b-9a482989a051_26,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [21]:
#Save the best model
best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')
joblib.dump(fitted_model, filename= "outputs/automl_model.pkl")

['outputs/automl_model.pkl']

## Model Deployment

In [22]:
# Download score.py and env file
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
best_run.download_file(constants.CONDA_ENV_FILE_PATH,'automl_env.yml')

In [23]:
# Register the model
model_name = best_run.properties['model_name']
description = 'AutoML Model trained on heart failure data to predict if death event occurs or not'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)

print(remote_run.model_id) # This will be written to the script file later in the notebook.

AutoML3e325779f26


In [25]:
# Create inference config
script_file_name= 'inference/score.py'
inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "hfData", 'type': "automl_classification"}, 
                                               description = 'Heart Failure Prediction')

aci_service_name = 'automl-heart-failure'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automl-heart-failure
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [26]:
# Enable Application Insights
aci_service.update(enable_app_insights=True)

In [27]:
print("State "+ aci_service.state)
print("Swagger URI " + aci_service.swagger_uri)
print("Scoring URI " + aci_service.scoring_uri)

State Healthy
Swagger URI http://c18b094b-8e53-466f-9940-747a62b057b6.southcentralus.azurecontainer.io/swagger.json
Scoring URI http://c18b094b-8e53-466f-9940-747a62b057b6.southcentralus.azurecontainer.io/score


In the cell below, a request is sent to the web service deployed to test it.

In [28]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = aci_service.scoring_uri
# If the service is authenticated, set the key or token

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
            "age": 70.0,
            "anaemia": 1,
            "creatinine_phosphokinase": 4020,
            "diabetes": 1,
            "ejection_fraction": 32,
            "high_blood_pressure": 1,
            "platelets": 234558.23,
            "serum_creatinine": 1.4,
            "serum_sodium": 125,
            "sex": 0,
            "smoking": 1,
            "time": 12
          },
          {
            "age": 75.0,
            "anaemia": 0,
            "creatinine_phosphokinase": 4221,
            "diabetes": 0,
            "ejection_fraction": 22,
            "high_blood_pressure": 0,
            "platelets": 404567.23,
            "serum_creatinine": 1.1,
            "serum_sodium": 115,
            "sex": 1,
            "smoking": 0,
            "time": 7
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": [1, 1]}


In [29]:
#Print the logs of the deployed service
aci_service.get_logs()

'2021-02-11T15:10:24,605347107+00:00 - iot-server/run \n2021-02-11T15:10:24,607125311+00:00 - rsyslog/run \n2021-02-11T15:10:24,616079537+00:00 - nginx/run \n2021-02-11T15:10:24,616498662+00:00 - gunicorn/run \nrsyslogd: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libuuid.so.1: no version information available (required by rsyslogd)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx

In [30]:
#Delete the deployed service
aci_service.delete()